In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [10]:
# initial DB of test samples including nuclide vectors
sfco_pkl = '~/sfcompo/format_clean/sfcompo_formatted.pkl'
sfcoDB = pd.read_pickle(sfco_pkl)

# results DBs with LogLikelihoods and predictions
sfco_path = '~/sims_n_results/nucmoles_opusupdate_aug2019/sfco/'
uncs = [0.05, 0.1, 0.15, 0.2]
job_dirs = ['Job' + str(i) + '_unc' + str(unc) for i, unc in enumerate(uncs)]
sfcoLL = {}
for i, unc_job in enumerate(job_dirs):
    sfcoLL[str(i)] = pd.read_csv(sfco_path + unc_job + '/' + unc_job + '.csv')

In [11]:
for unc_df in list(sfcoLL.keys()):
    df = sfcoLL[unc_df]
    df['Percent_Burnup_Error'] = df['Burnup_Error'] / df['Burnup']
    #df['AdjDiff'] = df['MaxLogLL'] - df['2ndMaxLogLL']
    #df['X'] = df['AdjDiff'] / np.log(2)

In [12]:
sfcoLL['0'].columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'sim_idx', 'ReactorType', 'CoolingTime',
       'Enrichment', 'Burnup', 'OrigenReactor', 'pred_idx', 'pred_ReactorType',
       'pred_CoolingTime', 'pred_Enrichment', 'pred_Burnup',
       'pred_OrigenReactor', 'MaxLogLL', 'MaxLLUnc', '2ndMaxLogLL',
       '2ndMaxLLUnc', 'CDF_LogLL_0.9998', 'CDF_LLUnc_0.9998',
       'CDF_LogLL_0.9988', 'CDF_LLUnc_0.9988', 'CDF_LogLL_0.95',
       'CDF_LLUnc_0.95', 'CDF_LogLL_0.9', 'CDF_LLUnc_0.9', 'CDF_LogLL_0.5',
       'CDF_LLUnc_0.5', 'CDF_LogLL_0.1', 'CDF_LLUnc_0.1', 'CDF_LogLL_0.01',
       'CDF_LLUnc_0.01', 'ReactorType_Score', 'CoolingTime_Error',
       'Enrichment_Error', 'Burnup_Error', 'OrigenReactor_Score',
       'Percent_Burnup_Error'],
      dtype='object')

In [80]:
sfcoDB.columns

Index(['Burnup', 'CoolingTime', 'Enrichment', 'OrigenReactor', 'ReactorType',
       'ba136', 'ba138', 'cs133', 'cs134', 'cs135', 'cs137', 'eu153', 'eu154',
       'pu239', 'pu240', 'pu241', 'pu242', 'sm149', 'sm150', 'sm152'],
      dtype='object')

In [81]:
lbls = ['ReactorType', 'CoolingTime', 'Enrichment', 'Burnup', 'OrigenReactor']
ratios = sfcoDB.loc[:, lbls].copy()

#cs137/cs133
ratios['cs137/cs133'] = sfcoDB['cs137'] / sfcoDB['cs133']
#cs134/cs137
ratios['cs134/cs137'] = sfcoDB['cs134'] / sfcoDB['cs137']
#cs135/cs137
ratios['cs135/cs137'] = sfcoDB['cs135'] / sfcoDB['cs137']
#ba136/ba138
ratios['ba136/ba138'] = sfcoDB['ba136'] / sfcoDB['ba138']
#sm150/sm149
ratios['sm150/sm149'] = sfcoDB['sm150'] / sfcoDB['sm149']
#sm152/sm149
ratios['sm152/sm149'] = sfcoDB['sm152'] / sfcoDB['sm149']
#eu154/eu153
ratios['eu154/eu153'] = sfcoDB['eu154'] / sfcoDB['eu153']
#pu240/pu239
ratios['pu240/pu239'] = sfcoDB['pu240'] / sfcoDB['pu239']
#pu241/pu239
ratios['pu241/pu239'] = sfcoDB['pu241'] / sfcoDB['pu239']
#pu242/pu239
ratios['pu242/pu239'] = sfcoDB['pu242'] / sfcoDB['pu239']

ratios.replace([np.inf, -np.inf], 0, inplace=True)
ratios.fillna(0, inplace = True)

In [82]:
ratios.columns

Index(['ReactorType', 'CoolingTime', 'Enrichment', 'Burnup', 'OrigenReactor',
       'cs137/cs133', 'cs134/cs137', 'cs135/cs137', 'ba136/ba138',
       'sm150/sm149', 'sm152/sm149', 'eu154/eu153', 'pu240/pu239',
       'pu241/pu239', 'pu242/pu239'],
      dtype='object')

In [83]:
sfcoLL['0'].shape, ratios.shape

((544, 38), (544, 15))

In [84]:
to_print = ['sim_idx', 'ReactorType', 'Enrichment', 'Burnup', 'pred_idx', 'pred_ReactorType', 'pred_Enrichment', 'pred_Burnup', 'ReactorType_Score', 'Enrichment_Error', 'Burnup_Error', 'Percent_Burnup_Error']
hundo_error = sfcoLL['0'].loc[sfcoLL['0']['Percent_Burnup_Error'] > 1.0, to_print]
hundo_error

,sim_idx,ReactorType,Enrichment,Burnup,pred_idx,pred_ReactorType,pred_Enrichment,pred_Burnup,ReactorType_Score,Enrichment_Error,Burnup_Error,Percent_Burnup_Error
36,FDN-2|2F2DN23|SF98|1,bwr,0.710,4150.0,11243,bwr,0.500,10045.28,True,0.210,5895.28,1.420549
59,FDN-2|2F2DN23|SF99|10,bwr,0.710,7190.0,46,pwr,0.530,15086.61,False,0.180,7896.61,1.098277
267,NPD-1|1022|A|1,phwr,0.711,791.0,20194,bwr,1.360,2961.22,False,0.649,2170.22,2.743641
268,NPD-1|1022|B|1,phwr,0.711,854.0,14605,bwr,2.160,2826.34,False,1.449,1972.34,2.309532
269,NPD-1|1022|C|1,phwr,0.711,1059.0,14605,bwr,2.160,2826.34,False,1.449,1767.34,1.668876
270,NPD-1|1129|A|1,phwr,0.711,1223.0,18736,bwr,1.620,3244.15,False,0.909,2021.15,1.652617
271,NPD-1|1129|B|1,phwr,0.711,1306.0,196,pwr,1.410,3609.51,False,0.699,2303.51,1.763790
272,NPD-1|1129|C|1,phwr,0.711,1622.0,17287,bwr,1.470,4174.70,False,0.759,2552.70,1.573798
273,NPD-1|1630|A|1,phwr,0.711,2733.0,17052,bwr,0.540,5915.89,False,0.171,3182.89,1.164614
274,NPD-1|1630|B|1,phwr,0.711,2733.0,24,pwr,0.530,6414.74,False,0.181,3681.74,1.347142


In [85]:
# worst Burnup pred: sim_idx is NPD-1|1022|A|1
ratios.loc['NPD-1|922|A|1']

ReactorType                                             phwr
CoolingTime                                                0
Enrichment                                             0.711
Burnup                                                  3809
OrigenReactor    Nuclear Power Demonstration Reactor-1_CANDU
cs137/cs133                                                0
cs134/cs137                                                0
cs135/cs137                                                0
ba136/ba138                                                0
sm150/sm149                                                0
sm152/sm149                                                0
eu154/eu153                                                0
pu240/pu239                                         0.215892
pu241/pu239                                        0.0313633
pu242/pu239                                        0.0042531
Name: NPD-1|922|A|1, dtype: object

In [86]:
ratios.head()

,ReactorType,CoolingTime,Enrichment,Burnup,OrigenReactor,cs137/cs133,cs134/cs137,cs135/cs137,ba136/ba138,sm150/sm149,sm152/sm149,eu154/eu153,pu240/pu239,pu241/pu239,pu242/pu239
BAL-2|1476|42|15,pwr,0.0,4.4,45100.0,Balakovo-2_VVER-1000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.425041,0.280066,0.113674
BAL-2|1476|42|31,pwr,0.0,4.4,45800.0,Balakovo-2_VVER-1000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.428110,0.274637,0.113086
BAL-2|1476|42|6,pwr,0.0,4.4,46700.0,Balakovo-2_VVER-1000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.437705,0.288525,0.118033
BAL-3|1591|23|912,pwr,0.0,4.4,46200.0,Balakovo-3_VVER-1000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.418530,0.281150,0.119808
BAL-3|1591|23|581,pwr,0.0,4.4,47900.0,Balakovo-3_VVER-1000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.440789,0.289474,0.144737


In [88]:
# do not rerun
ratios['> zero'] = ratios[ratios.columns[~ratios.columns.isin(lbls)]].gt(0).sum(axis=1)
ratios['== zero'] = ratios[ratios.columns[~ratios.columns.isin(lbls)]].eq(0).sum(axis=1)
ratios

,ReactorType,CoolingTime,Enrichment,Burnup,OrigenReactor,cs137/cs133,cs134/cs137,cs135/cs137,ba136/ba138,sm150/sm149,sm152/sm149,eu154/eu153,pu240/pu239,pu241/pu239,pu242/pu239,> zero,== zero
BAL-2|1476|42|15,pwr,0.0,4.4,45100.0,Balakovo-2_VVER-1000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.425041,0.280066,0.113674,3,7
BAL-2|1476|42|31,pwr,0.0,4.4,45800.0,Balakovo-2_VVER-1000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.428110,0.274637,0.113086,3,7
BAL-2|1476|42|6,pwr,0.0,4.4,46700.0,Balakovo-2_VVER-1000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.437705,0.288525,0.118033,3,7
BAL-3|1591|23|912,pwr,0.0,4.4,46200.0,Balakovo-3_VVER-1000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.418530,0.281150,0.119808,3,7
BAL-3|1591|23|581,pwr,0.0,4.4,47900.0,Balakovo-3_VVER-1000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.440789,0.289474,0.144737,3,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
YNK-1|E6|NW-A1|G-127,pwr,0.0,3.4,42500.0,Yankee-1_PWR,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.426646,0.294930,0.139970,3,7
YNK-1|E6|NW-A1|G-128,pwr,0.0,3.4,42500.0,Yankee-1_PWR,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.436314,0.301077,0.137288,3,7
YNK-1|E6|NW-A1|N-21,pwr,0.0,3.4,42600.0,Yankee-1_PWR,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.420678,0.284631,0.122891,3,7
YNK-1|E6|NW-A1|T-165,pwr,0.0,3.4,43200.0,Yankee-1_PWR,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.432166,0.291983,0.132763,3,7


In [62]:
count = ratios.loc[ratios['> zero'] == 3]
count

,ReactorType,CoolingTime,Enrichment,Burnup,OrigenReactor,cs137/cs133,cs134/cs137,cs135/cs137,ba136/ba138,sm150/sm149,sm152/sm149,eu154/eu153,pu240/pu239,pu241/pu239,pu242/pu239,> zero,== zero
NOV-5|4433001114|40|34,pwr,0.0,4.400,50200.000000,Novovoronezh-5_VVER-1000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.464041,0.280822,0.142808,3,7
TVC-1|509-104|A12|1,pwr,0.0,3.897,3399.000000,Trino Vercellese-1_PWR,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.063251,0.008060,0.000273,3,7
OBR-1|BE124|E3|P2,pwr,0.0,3.000,35100.000000,Obrigheim-1_PWR,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.430736,0.220779,0.082251,3,7
MON-1|MTB099|A1|S5,bwr,0.0,1.450,46400.000000,Monticello-1_BWR,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.823529,0.334754,0.628588,3,7
OBR-1|BE124|G7|P2,pwr,0.0,3.000,25800.000000,Obrigheim-1_PWR,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.385106,0.200000,0.059362,3,7
FDN-2|2F2D1|B3|TU105,bwr,0.0,3.400,12300.000000,Fukushima-Daini-2_BWR,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.206897,0.081902,0.008621,3,7
YNK-1|E6|C-A1|G-108,pwr,0.0,3.400,30000.000000,Yankee-1_PWR,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.294101,0.208264,0.051892,3,7
FDN-1|2F1ZN3|C3|UT,bwr,0.0,4.900,59050.000000,Fukushima-Daini-1_BWR,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.628072,0.262865,0.185810,3,7
NPD-1|1022|C|1,phwr,0.0,0.711,1059.000000,Nuclear Power Demonstration Reactor-1_CANDU,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.065304,0.004946,0.000273,3,7
NPD-1|1092|C|1,phwr,0.0,0.711,9800.000000,Nuclear Power Demonstration Reactor-1_CANDU,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.480008,0.103155,0.039490,3,7


In [91]:
ratios['> zero'].describe()

count    544.000000
mean       3.867647
std        1.722172
min        2.000000
25%        3.000000
50%        3.000000
75%        4.000000
max        9.000000
Name: > zero, dtype: float64

In [64]:
allIsos = pd.read_pickle('~/sfcompo/format_clean/sfcompo_all_isos_formatted.pkl')
allIsos

,Burnup,CoolingTime,Enrichment,OrigenReactor,ReactorType,ag109,ag110m,am,am241,am242,...,sm154,sr90,tc99,u,u232,u233,u234,u235,u236,u238
BAL-2|1476|42|15,45100.0,0.0,4.4,Balakovo-2_VVER-1000,pwr,0.0,0.0,0.0,0.048,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000,10.640,6.170,926.55
BAL-2|1476|42|31,45800.0,0.0,4.4,Balakovo-2_VVER-1000,pwr,0.0,0.0,0.0,0.000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.226,10.660,6.000,925.00
BAL-2|1476|42|6,46700.0,0.0,4.4,Balakovo-2_VVER-1000,pwr,0.0,0.0,0.0,0.066,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000,10.270,6.110,925.18
BAL-3|1591|23|912,46200.0,0.0,4.4,Balakovo-3_VVER-1000,pwr,0.0,0.0,0.0,0.054,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000,10.180,6.150,925.48
BAL-3|1591|23|581,47900.0,0.0,4.4,Balakovo-3_VVER-1000,pwr,0.0,0.0,0.0,0.071,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000,8.180,6.340,925.66
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
YNK-1|E6|NW-A1|G-127,42500.0,0.0,3.4,Yankee-1_PWR,pwr,0.0,0.0,0.0,0.000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.111,7.249,4.770,932.06
YNK-1|E6|NW-A1|G-128,42500.0,0.0,3.4,Yankee-1_PWR,pwr,0.0,0.0,0.0,0.000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.105,7.125,4.598,932.39
YNK-1|E6|NW-A1|N-21,42600.0,0.0,3.4,Yankee-1_PWR,pwr,0.0,0.0,0.0,0.000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.102,7.373,4.569,931.37
YNK-1|E6|NW-A1|T-165,43200.0,0.0,3.4,Yankee-1_PWR,pwr,0.0,0.0,0.0,0.000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.104,7.020,4.703,931.16


In [89]:
# do not rerun
allIsos['> zero'] = allIsos[allIsos.columns[~allIsos.columns.isin(lbls)]].gt(0).sum(axis=1)
allIsos['== zero'] = allIsos[allIsos.columns[~allIsos.columns.isin(lbls)]].eq(0).sum(axis=1)
allIsos

,Burnup,CoolingTime,Enrichment,OrigenReactor,ReactorType,ag109,ag110m,am,am241,am242,...,tc99,u,u232,u233,u234,u235,u236,u238,> zero,== zero
BAL-2|1476|42|15,45100.0,0.0,4.4,Balakovo-2_VVER-1000,pwr,0.0,0.0,0.0,0.048,0.0,...,0.0,0.0,0.0,0.0,0.000,10.640,6.170,926.55,17,68
BAL-2|1476|42|31,45800.0,0.0,4.4,Balakovo-2_VVER-1000,pwr,0.0,0.0,0.0,0.000,0.0,...,0.0,0.0,0.0,0.0,0.226,10.660,6.000,925.00,13,72
BAL-2|1476|42|6,46700.0,0.0,4.4,Balakovo-2_VVER-1000,pwr,0.0,0.0,0.0,0.066,0.0,...,0.0,0.0,0.0,0.0,0.000,10.270,6.110,925.18,17,68
BAL-3|1591|23|912,46200.0,0.0,4.4,Balakovo-3_VVER-1000,pwr,0.0,0.0,0.0,0.054,0.0,...,0.0,0.0,0.0,0.0,0.000,10.180,6.150,925.48,17,68
BAL-3|1591|23|581,47900.0,0.0,4.4,Balakovo-3_VVER-1000,pwr,0.0,0.0,0.0,0.071,0.0,...,0.0,0.0,0.0,0.0,0.000,8.180,6.340,925.66,17,68
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
YNK-1|E6|NW-A1|G-127,42500.0,0.0,3.4,Yankee-1_PWR,pwr,0.0,0.0,0.0,0.000,0.0,...,0.0,0.0,0.0,0.0,0.111,7.249,4.770,932.06,10,75
YNK-1|E6|NW-A1|G-128,42500.0,0.0,3.4,Yankee-1_PWR,pwr,0.0,0.0,0.0,0.000,0.0,...,0.0,0.0,0.0,0.0,0.105,7.125,4.598,932.39,10,75
YNK-1|E6|NW-A1|N-21,42600.0,0.0,3.4,Yankee-1_PWR,pwr,0.0,0.0,0.0,0.000,0.0,...,0.0,0.0,0.0,0.0,0.102,7.373,4.569,931.37,8,77
YNK-1|E6|NW-A1|T-165,43200.0,0.0,3.4,Yankee-1_PWR,pwr,0.0,0.0,0.0,0.000,0.0,...,0.0,0.0,0.0,0.0,0.104,7.020,4.703,931.16,9,76


In [90]:
allIsos['> zero'].describe()

count    544.000000
mean      17.608456
std       11.805795
min        5.000000
25%        9.000000
50%       12.000000
75%       26.000000
max       66.000000
Name: > zero, dtype: float64